In [ ]:
import numpy as np
import pandas as pd

from my_utilities import *
from Make_QSO_altered_2 import add_errors

import matplotlib.pyplot as plt

In [ ]:
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u

Hubble_h = 0.73
Omega_M = 0.25
Omega_Lambda = 0.75
cosmo = FlatLambdaCDM(H0=Hubble_h*100, Om0=Omega_M)

In [ ]:
lc = np.load('/home/alberto/almacen/David_lightcone/LightCone_512_15sep2022.npy')

In [ ]:
def M_to_m(M, redshift, x, y, z):
    '''
    Computes the apparent magnitude from the absolute magnitude
    Inputs:
    M: Absolute magnitude
    redshift: Redshift of the source
    x, y, z: Coordinates in the Lightcone (pc)
    '''
    # Luminosity distance:
    dL = cosmo.luminosity_distance(redshift).to(u.pc).value.reshape(-1, 1)

    return M + 5 * np.log10(dL) - 5

In [ ]:
w_central = central_wavelength()

In [ ]:
def M_Arr_to_pm_flx(M_Arr, z, XX, YY, ZZ):
    m = M_to_m(M_Arr, z, XX, YY, ZZ).T
    flx = mag_to_flux(m, w_central.reshape(-1, 1))
    return flx

In [ ]:
xx, yy, zz = lc['pos'].T
z = lc['redshift']
Rdisk = lc['Rdisk']
Rbulge = lc['Rbulge']
Mdisk = lc['DiskMass']
Mbulge = lc['BulgeMass']

OII_flx_dust = (lc['FluxDust_OII3727'] + lc['FluxDust_OII3729'])
OII_flx_nodust = (lc['Flux_OII3727'] + lc['Flux_OII3729'])

OII_flx_correction = OII_flx_nodust / OII_flx_dust
OII_flx_correction[~np.isfinite(OII_flx_correction)] = 1.

pm_flx_lc = M_Arr_to_pm_flx(lc['ObsMagDustLine'], z, xx, yy, zz)
pm_flx_c = M_Arr_to_pm_flx(lc['ObsMagDustNoLine'], z, xx, yy, zz)
pm_flx_l = pm_flx_lc - pm_flx_c

In [ ]:
# Mask OII lines
w_0_Mat = np.ones_like(pm_flx_lc) * w_central.reshape(-1, 1) / (1 + z)
w_0_OII = (3727.092 + 3729.875) * 0.5
dw = 145
mask_OII = np.abs(w_0_Mat - w_0_OII) < dw
OII_flx_corr_Mat = np.ones_like(mask_OII) * OII_flx_correction
OII_flx_corr_Mat[~mask_OII] = 1.

In [ ]:
pm_flx_lc = pm_flx_c + (pm_flx_l * OII_flx_corr_Mat)

In [ ]:
print(np.sum(pm_mag_linecont[-2] < 24))

fig, ax = plt.subplots(figsize=(6, 3))

ax.hist(pm_mag_linecont[-2], np.linspace(15, 27, 50), histtype='stepfilled',
        linewidth=1.5, edgecolor='k', facecolor='lightgray')

ax.set_xlim(15, 26)
ax.set_yscale('log')

ax.set_ylabel('N of objects', fontsize=15)
ax.set_xlabel('r [magAB]', fontsize=15)

fig.savefig('/home/alberto/Desktop/LC_r_dist.pdf', bbox_inches='tight', facecolor='w')
plt.show()

fig, ax = plt.subplots(figsize=(6, 3))

ax.hist(z, np.linspace(0, 5, 50), histtype='stepfilled',
        linewidth=1.5, edgecolor='k', facecolor='lightgray')

# ax.set_xlim(15, 26)
# ax.set_yscale('log')

ax.set_ylabel('N of objects', fontsize=15)
ax.set_xlabel('redshift', fontsize=15)

fig.savefig('/home/alberto/Desktop/LC_z_dist.pdf', bbox_inches='tight', facecolor='w')
plt.show()

In [ ]:
# pm_flx_noline, pm_err = add_errors(pm_flx_noline, apply_err=False, survey_name='minijpas')

tcurves = np.load('../npy/tcurves.npy', allow_pickle=True).item()
ftags = tcurves['tag']
hdr = (ftags + [s + '_e' for s in ftags] + ['z', 'Rdisk', 'Rbulge', 'Mdisk', 'Mbulge'])

data = np.vstack([pm_flx_lc, pm_flx_lc * 0., z, Rdisk, Rbulge, Mdisk, Mbulge]).T

filename = f'/home/alberto/almacen/Source_cats/GAL_LC_lines_0.csv'
pd.DataFrame(data=data).to_csv(filename, header=hdr)